# Script de Carregamento, Pré-processamento e Aumento de Dados

O script a seguir utiliza TensorFlow/Keras para carregar as imagens organizadas na pasta chars74k_dataset_final, aplicar o pré-processamento necessário (grayscale, redimensionamento, normalização) e incluir o Aumento de Dados (Data Augmentation) apenas para o conjunto de treinamento.

Este é o método ideal para garantir que o modelo CNN receba dados consistentes e robustos.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# --- 1. CONFIGURAÇÕES GERAIS ---
# Caminho para o dataset organizado
DATASET_PATH = 'chars74k_dataset_final'

# Tamanho alvo para redimensionamento
IMAGE_SIZE = (64, 64) 

# Tamanho do lote (batch) para treinamento
BATCH_SIZE = 32

# Número de classes (62 classes: 10 dígitos + 26 maiúsculas + 26 minúsculas)
NUM_CLASSES = 62

# --- 2. CONFIGURAÇÃO DE AUMENTO DE DADOS (DATA AUGMENTATION) ---

# O Data Augmentation deve ser aplicado APENAS ao conjunto de TREINAMENTO
train_datagen = ImageDataGenerator(
    # Normalização dos pixels para o intervalo [0, 1] (Dividir por 255)
    rescale=1./255,
    # Técnicas de Augmentation:
    rotation_range=10,        # Rotação aleatória (máx. 10 graus)
    width_shift_range=0.1,    # Deslocamento horizontal (até 10% da largura)
    height_shift_range=0.1,   # Deslocamento vertical (até 10% da altura)
    zoom_range=0.1,           # Zoom aleatório
    shear_range=0.1,          # Cisalhamento
    fill_mode='nearest',      # Estratégia para preencher novos pixels após transformações
    # horizontal_flip=True    # Evitado para caracteres, pois 'p' e 'd' podem se confundir
)

# Para Validação e Teste, fazemos APENAS a Normalização (rescale)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# --- 3. CARREGAMENTO E PRÉ-PROCESSAMENTO DOS DADOS ---

print("Carregando e pré-processando dados de Treinamento...")
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'train'),
    target_size=IMAGE_SIZE,
    color_mode='grayscale',           # CONVERTE IMAGENS PARA ESCALA DE CINZA
    batch_size=BATCH_SIZE,
    class_mode='categorical',         # Necessário para 62 classes (one-hot encoding)
    save_format='png'                 # Mantém o formato PNG (apenas durante o carregamento)
)

print("\nCarregando e pré-processando dados de Validação...")
validation_generator = val_test_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'validation'),
    target_size=IMAGE_SIZE,
    color_mode='grayscale',           # CONVERTE IMAGENS PARA ESCALA DE CINZA
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("\nCarregando e pré-processando dados de Teste...")
test_generator = val_test_datagen.flow_from_directory(
    os.path.join(DATASET_PATH, 'test'),
    target_size=IMAGE_SIZE,
    color_mode='grayscale',           # CONVERTE IMAGENS PARA ESCALA DE CINZA
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False                     # Desativa o shuffle para manter a ordem para avaliação final
)

# --- 4. VERIFICAÇÃO E INFORMAÇÕES ADICIONAIS ---

print("\n--- Informações do Dataset ---")
print(f"Total de classes identificadas: {len(train_generator.class_indices)}")
print(f"Mapeamento de classes (Exemplo): {list(train_generator.class_indices.items())[:10]}...") 

# Exemplo de como um batch de dados aparece
X, y = next(train_generator)
print(f"Formato de um lote de Treinamento (X): {X.shape}") # Deve ser (BATCH_SIZE, 64, 64, 1)
print(f"Formato de um lote de Rótulos (y): {y.shape}")     # Deve ser (BATCH_SIZE, NUM_CLASSES)
print(f"Tipo de dados: {X.dtype}")

# ----------------------------------------------------
# Onde `train_generator`, `validation_generator` e `test_generator` 
# estão prontos para serem passados para o método model.fit().
# ----------------------------------------------------

2025-11-20 06:16:49.380851: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-20 06:16:57.792451: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 06:17:10.495797: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Carregando e pré-processando dados de Treinamento...
Found 51804 images belonging to 62 classes.

Carregando e pré-processando dados de Validação...
Found 7367 images belonging to 62 classes.

Carregando e pré-processando dados de Teste...
Found 7371 images belonging to 62 classes.

--- Informações do Dataset ---
Total de classes identificadas: 62
Mapeamento de classes (Exemplo): [('0', 0), ('1', 1), ('2', 2), ('3', 3), ('4', 4), ('5', 5), ('6', 6), ('7', 7), ('8', 8), ('9', 9)]...
Formato de um lote de Treinamento (X): (32, 64, 64, 1)
Formato de um lote de Rótulos (y): (32, 62)
Tipo de dados: float32
